In [ ]:
import os
import json
from tqdm import tqdm
from collections import defaultdict, Counter
from termcolor import colored
import lib.time_logger as TLOG

# Define paths (you will need to set these in your notebook)
def success(msg):
    """Print success message in green"""
    tqdm.write(colored(f"✓ SUCCESS: {msg}", "green"))

def error(msg):
    """Print error message in red"""
    tqdm.write(colored(f"✗ ERROR: {msg}", "red"))

def info(msg):
    """Print info message in blue"""
    tqdm.write(colored(f"ℹ INFO: {msg}", "blue"))

def warning(msg):
    """Print warning message in yellow"""
    tqdm.write(colored(f"⚠ WARNING: {msg}", "yellow"))

def extract_setup_key(metadata):
    """Extract only the setup parameters from metadata, excluding strategies."""
    setup_params = {}
    for key, value in metadata.items():
        if key not in ['attacker_strategy', 'defender_strategy']:
            setup_params[key] = value
    return json.dumps(setup_params, sort_keys=True)

def process_data_directory(data_path):
    """Process all JSON files in a directory and return a dictionary of results."""
    # Get all JSON files in the data directory
    json_files = [filename for filename in os.listdir(data_path) if filename.endswith(".json")]
    info(f"Found {len(json_files)} JSON files in {data_path}")
    
    data_dict = {}
    
    pbar = tqdm(json_files, desc=f"Processing {data_path}", unit="file", ncols=200)
    
    # For each JSON file, extract the hash key in the filename
    for filename in pbar:
        try:
            # Extract the hash key from the filename
            hash_key = (filename.split(".")[0]).split("_")[1]
            pbar.set_postfix_str(f"Processing {hash_key}")
            
            # Read the file directly instead of using logger.extract_summary()
            file_path = os.path.join(data_path, filename)
            with open(file_path, 'r') as f:
                file_data = json.load(f)
            
            # Extract metadata
            metadata = file_data.get("metadata", {})
            if "attacker_strategy" not in metadata or "defender_strategy" not in metadata:
                warning(f"File {filename} is missing strategy information. Skipping.")
                continue
            
            # Extract the summary data from the last record
            records = file_data.get("records", [])
            summary = None
            for record in reversed(records):  # Search from the end
                if record.get("summary") == True:
                    summary = record
                    break
            
            if not summary:
                warning(f"File {filename} does not contain summary data. Skipping.")
                continue
            
            # Check for error message
            has_error = "error" in summary
            
            data_dict[filename] = {
                "hash_key": hash_key,
                "attacker_strategy": metadata.get("attacker_strategy"),
                "defender_strategy": metadata.get("defender_strategy"),
                "parameters": extract_setup_key(metadata),
                "summary": summary,
                "has_error": has_error
            }
        except Exception as e:
            error(f"Error processing {filename}: {str(e)}")
        
        pbar.update(1)
    
    return data_dict

def analyze_strategy_frequencies(data_dict, file_list):
    """Analyze how often each strategy appears in the given file list."""
    attacker_strategies = Counter()
    defender_strategies = Counter()
    
    for filename in file_list:
        if filename in data_dict:
            file_data = data_dict[filename]
            attacker_strategies[file_data['attacker_strategy']] += 1
            defender_strategies[file_data['defender_strategy']] += 1
    
    return {
        'attacker_strategies': attacker_strategies,
        'defender_strategies': defender_strategies
    }

def print_strategy_frequencies(strategy_counters, category_name):
    """Print the strategy frequency information in a readable format."""
    info(f"\nStrategy frequency analysis for {category_name}:")
    
    attacker_strategies = strategy_counters['attacker_strategies']
    defender_strategies = strategy_counters['defender_strategies']
    
    if attacker_strategies:
        info("Attacker strategies:")
        for strategy, count in sorted(attacker_strategies.items(), key=lambda x: x[1], reverse=True):
            info(f"  - {strategy}: {count} occurrences")
    else:
        info("No attacker strategies found.")
        
    if defender_strategies:
        info("Defender strategies:")
        for strategy, count in sorted(defender_strategies.items(), key=lambda x: x[1], reverse=True):
            info(f"  - {strategy}: {count} occurrences")
    else:
        info("No defender strategies found.")

def compare_repositories(data_path_1, data_path_2):
    """Compare two repositories of JSON files."""
    info("Starting comparison between repositories")
    
    # Process both directories
    data_dict_1 = process_data_directory(data_path_1)
    data_dict_2 = process_data_directory(data_path_2)
    
    # Find files in repo 1 but not in repo 2
    missing_in_repo2 = set(data_dict_1.keys()) - set(data_dict_2.keys())
    
    # Find files in repo 2 but not in repo 1
    missing_in_repo1 = set(data_dict_2.keys()) - set(data_dict_1.keys())
    
    # Find files with errors
    files_with_errors_1 = [f for f, data in data_dict_1.items() if data["has_error"]]
    files_with_errors_2 = [f for f, data in data_dict_2.items() if data["has_error"]]
    
    # Find common files with different results
    common_files = set(data_dict_1.keys()).intersection(set(data_dict_2.keys()))
    different_results = []
    
    pbar = tqdm(common_files, desc="Comparing common files", unit="file", ncols=200)
    for filename in pbar:
        pbar.set_postfix_str(f"Comparing {filename}")
        summary1 = data_dict_1[filename]["summary"]
        summary2 = data_dict_2[filename]["summary"]
        
        # Compare summary fields
        if (summary1.get("payoff") != summary2.get("payoff") or
            summary1.get("time") != summary2.get("time") or
            summary1.get("total_captures") != summary2.get("total_captures") or
            summary1.get("total_tags") != summary2.get("total_tags")):
            different_results.append({
                "filename": filename,
                "repo1": summary1,
                "repo2": summary2
            })
    
    # Convert list of dict to set of filenames for strategy analysis
    different_files = {item["filename"] for item in different_results}
    
    # Analyze strategy frequencies for different categories
    missing_repo2_strategies = analyze_strategy_frequencies(data_dict_1, missing_in_repo2)
    missing_repo1_strategies = analyze_strategy_frequencies(data_dict_2, missing_in_repo1)
    errors_repo1_strategies = analyze_strategy_frequencies(data_dict_1, files_with_errors_1)
    errors_repo2_strategies = analyze_strategy_frequencies(data_dict_2, files_with_errors_2)
    different_results_strategies = analyze_strategy_frequencies(data_dict_1, different_files)
    
    # Print summary
    info("=" * 50)
    info("Comparison Summary")
    info("=" * 50)
    
    # Summary statistics
    info(f"\nTotal files in Repo 1: {len(data_dict_1)}")
    info(f"Total files in Repo 2: {len(data_dict_2)}")
    info(f"Files only in Repo 1: {len(missing_in_repo2)}")
    info(f"Files only in Repo 2: {len(missing_in_repo1)}")
    info(f"Files with errors in Repo 1: {len(files_with_errors_1)}")
    info(f"Files with errors in Repo 2: {len(files_with_errors_2)}")
    info(f"Files with different results: {len(different_results)}")
    info(f"Files with matching results: {len(common_files) - len(different_results)}")
    
    # Detailed file listings
    if missing_in_repo2:
        warning(f"\nFound {len(missing_in_repo2)} files in Repo 1 that are missing in Repo 2")
        for f in list(missing_in_repo2)[:10]:  # Show first 10
            warning(f"  Missing in Repo 2: {f}")
        if len(missing_in_repo2) > 10:
            warning(f"  ... and {len(missing_in_repo2) - 10} more")
        # Print strategy frequencies
        print_strategy_frequencies(missing_repo2_strategies, "Files missing in Repo 2")
    else:
        success("All files in Repo 1 exist in Repo 2")
        
    if missing_in_repo1:
        warning(f"\nFound {len(missing_in_repo1)} files in Repo 2 that are missing in Repo 1")
        for f in list(missing_in_repo1)[:10]:  # Show first 10
            warning(f"  Missing in Repo 1: {f}")
        if len(missing_in_repo1) > 10:
            warning(f"  ... and {len(missing_in_repo1) - 10} more")
        # Print strategy frequencies
        print_strategy_frequencies(missing_repo1_strategies, "Files missing in Repo 1")
    else:
        success("All files in Repo 2 exist in Repo 1")
    
    if files_with_errors_1:
        error(f"\nFound {len(files_with_errors_1)} files with errors in Repo 1")
        for f in list(files_with_errors_1)[:10]:  # Show first 10
            error(f"  Error in Repo 1: {f}")
        if len(files_with_errors_1) > 10:
            error(f"  ... and {len(files_with_errors_1) - 10} more")
        # Print strategy frequencies
        print_strategy_frequencies(errors_repo1_strategies, "Files with errors in Repo 1")
    else:
        success("No files with errors in Repo 1")
        
    if files_with_errors_2:
        error(f"\nFound {len(files_with_errors_2)} files with errors in Repo 2")
        for f in list(files_with_errors_2)[:10]:  # Show first 10
            error(f"  Error in Repo 2: {f}")
        if len(files_with_errors_2) > 10:
            error(f"  ... and {len(files_with_errors_2) - 10} more")
        # Print strategy frequencies
        print_strategy_frequencies(errors_repo2_strategies, "Files with errors in Repo 2")
    else:
        success("No files with errors in Repo 2")
    
    if different_results:
        error(f"\nFound {len(different_results)} files with different results")
        for diff in different_results[:10]:  # Show first 10
            filename = diff['filename']
            repo1_summary = diff['repo1']
            repo2_summary = diff['repo2']
            error(f"  Different results: {filename}")
            error(f"    Repo 1: payoff={repo1_summary.get('payoff')}, time={repo1_summary.get('time')}, captures={repo1_summary.get('total_captures')}, tags={repo1_summary.get('total_tags')}")
            error(f"    Repo 2: payoff={repo2_summary.get('payoff')}, time={repo2_summary.get('time')}, captures={repo2_summary.get('total_captures')}, tags={repo2_summary.get('total_tags')}")
        if len(different_results) > 10:
            error(f"  ... and {len(different_results) - 10} more")
        # Print strategy frequencies
        print_strategy_frequencies(different_results_strategies, "Files with different results")
    else:
        success("All common files have matching results")
    
    # Return detailed stats for further analysis if needed
    return {
        "file_counts": {
            "repo1_total": len(data_dict_1),
            "repo2_total": len(data_dict_2),
            "missing_in_repo2": len(missing_in_repo2),
            "missing_in_repo1": len(missing_in_repo1),
            "with_errors_in_repo1": len(files_with_errors_1),
            "with_errors_in_repo2": len(files_with_errors_2),
            "with_different_results": len(different_results),
            "with_matching_results": len(common_files) - len(different_results)
        },
        "files": {
            "missing_in_repo2": missing_in_repo2,
            "missing_in_repo1": missing_in_repo1,
            "with_errors_in_repo1": files_with_errors_1,
            "with_errors_in_repo2": files_with_errors_2,
            "with_different_results": different_results
        },
        "strategy_frequencies": {
            "missing_in_repo2": missing_repo2_strategies,
            "missing_in_repo1": missing_repo1_strategies,
            "with_errors_in_repo1": errors_repo1_strategies,
            "with_errors_in_repo2": errors_repo2_strategies,
            "with_different_results": different_results_strategies
        }
    }

# Example usage in your notebook:
DATA_PATH_1 = os.path.join("data", "result_v1.0.1", "")
DATA_PATH_2 = os.path.join("data", "result_v1.1.1", "")
comparison_results = compare_repositories(DATA_PATH_1, DATA_PATH_2)

ℹ INFO: Starting comparison between repositories
ℹ INFO: Found 6400 JSON files in data/result/


Processing data/result/: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 6400/6400 [00:05<00:00, 1238.13file/s, Processing bdee3bca48]


ℹ INFO: Found 6400 JSON files in data/result-old/


Comparing common files: 100%|████████████████████████████████████████████████████| 6400/6400 [00:03<00:00, 1684.12file/s, Comparing log_815fb0dd95_attacker_MSU_Attacker_defender_Example_Defender.json]


ℹ INFO: ==================================================
ℹ INFO: Comparison Summary
ℹ INFO: ==================================================
ℹ INFO: 
Total files in Repo 1: 6400
ℹ INFO: Total files in Repo 2: 6400
ℹ INFO: Files only in Repo 1: 0
ℹ INFO: Files only in Repo 2: 0
ℹ INFO: Files with errors in Repo 1: 0
ℹ INFO: Files with errors in Repo 2: 0
ℹ INFO: Files with different results: 3991
ℹ INFO: Files with matching results: 2409
✓ SUCCESS: All files in Repo 1 exist in Repo 2
✓ SUCCESS: All files in Repo 2 exist in Repo 1
✓ SUCCESS: No files with errors in Repo 1
✓ SUCCESS: No files with errors in Repo 2
✗ ERROR: 
Found 3991 files with different results
✗ ERROR:   Different results: log_346e554198_attacker_GMU_Attacker_defender_UNCC_Defender.json
✗ ERROR:     Repo 1: payoff=-2.0, time=9, captures=2, tags=8
✗ ERROR:     Repo 2: payoff=0.0, time=9, captures=2, tags=4
✗ ERROR:   Different results: log_a6083ed2ca_attacker_GMU_Attacker_defender_Example_Defender.json
✗ ERROR:     

In [ ]:
import os
import json
from tqdm import tqdm
from collections import defaultdict, Counter
from termcolor import colored

def success(msg):
    """Print success message in green"""
    tqdm.write(colored(f"✓ SUCCESS: {msg}", "green"))

def error(msg):
    """Print error message in red"""
    tqdm.write(colored(f"✗ ERROR: {msg}", "red"))

def info(msg):
    """Print info message in blue"""
    tqdm.write(colored(f"ℹ INFO: {msg}", "blue"))

def warning(msg):
    """Print warning message in yellow"""
    tqdm.write(colored(f"⚠ WARNING: {msg}", "yellow"))

def analyze_stochastic_behavior(data_path_1, data_path_2, different_results):
    """
    Analyze which strategies are likely stochastic by checking patterns in differences.
    
    Args:
        data_path_1: Path to first repository
        data_path_2: Path to second repository
        different_results: List of dictionaries containing filenames with different results
    
    Returns:
        Dictionary with analysis of stochastic behaviors
    """
    info("Analyzing stochastic behavior in strategies...")
    
    # Convert list of dicts to set of filenames
    different_filenames = {item["filename"] for item in different_results}
    
    # Group differences by configuration and strategy pairs
    config_strategy_differences = defaultdict(list)
    strategy_pair_differences = defaultdict(list)
    
    for filename in different_filenames:
        # Load file data from both repos
        try:
            file_path_1 = os.path.join(data_path_1, filename)
            with open(file_path_1, 'r') as f:
                file_data_1 = json.load(f)
            
            metadata = file_data_1.get("metadata", {})
            attacker = metadata.get("attacker_strategy")
            defender = metadata.get("defender_strategy")
            
            # Create a hash key from all configuration parameters except strategies
            config_params = {k: v for k, v in metadata.items() 
                           if k not in ["attacker_strategy", "defender_strategy"]}
            config_key = json.dumps(config_params, sort_keys=True)
            
            # Store by configuration and strategy pair
            strategy_pair = f"{attacker} vs {defender}"
            config_strategy_differences[config_key].append(strategy_pair)
            strategy_pair_differences[strategy_pair].append(filename)
        except Exception as e:
            error(f"Error processing {filename} for stochastic analysis: {str(e)}")
    
    # Analyze which configurations have differences across multiple strategy pairs
    config_analysis = {}
    for config, strategy_pairs in config_strategy_differences.items():
        config_analysis[config] = {
            "unique_strategy_pairs": len(set(strategy_pairs)),
            "strategy_pairs": Counter(strategy_pairs)
        }
    
    # Calculate percentage of different results for each strategy
    strategy_consistency = analyze_strategy_consistency(data_path_1, different_filenames)
    
    # Identify strategies with highest variance in results
    stochastic_candidates = identify_stochastic_candidates(strategy_consistency)
    
    # Additional analysis for each strategy pair to find specific patterns
    strategy_pair_patterns = analyze_strategy_pair_patterns(different_results)
    
    return {
        "strategy_consistency": strategy_consistency,
        "stochastic_candidates": stochastic_candidates,
        "config_analysis": config_analysis,
        "strategy_pair_patterns": strategy_pair_patterns
    }

def analyze_strategy_consistency(data_path, different_filenames):
    """
    Analyze how consistent each strategy is by calculating the percentage
    of runs that differ when that strategy is used.
    """
    # Count total occurrences of each strategy
    total_attacker_counts = Counter()
    total_defender_counts = Counter()
    
    # Count different result occurrences for each strategy
    diff_attacker_counts = Counter()
    diff_defender_counts = Counter()
    
    # Process all files to count total strategy uses
    json_files = [f for f in os.listdir(data_path) if f.endswith('.json')]
    
    pbar = tqdm(json_files, desc="Analyzing strategy consistency", unit="file", ncols=100)
    for filename in pbar:
        try:
            file_path = os.path.join(data_path, filename)
            with open(file_path, 'r') as f:
                file_data = json.load(f)
            
            metadata = file_data.get("metadata", {})
            attacker = metadata.get("attacker_strategy")
            defender = metadata.get("defender_strategy")
            
            if attacker and defender:
                total_attacker_counts[attacker] += 1
                total_defender_counts[defender] += 1
                
                # Check if this file had different results
                if filename in different_filenames:
                    diff_attacker_counts[attacker] += 1
                    diff_defender_counts[defender] += 1
        except Exception as e:
            continue  # Skip problematic files
    
    # Calculate consistency percentages
    attacker_consistency = {}
    for attacker, total in total_attacker_counts.items():
        different = diff_attacker_counts.get(attacker, 0)
        consistency = 100 * (1 - (different / total)) if total > 0 else 100
        attacker_consistency[attacker] = {
            "total_runs": total,
            "different_runs": different,
            "consistency_percentage": consistency
        }
    
    defender_consistency = {}
    for defender, total in total_defender_counts.items():
        different = diff_defender_counts.get(defender, 0)
        consistency = 100 * (1 - (different / total)) if total > 0 else 100
        defender_consistency[defender] = {
            "total_runs": total,
            "different_runs": different,
            "consistency_percentage": consistency
        }
    
    return {
        "attacker_consistency": attacker_consistency,
        "defender_consistency": defender_consistency
    }

def identify_stochastic_candidates(strategy_consistency):
    """
    Identify which strategies are most likely to be stochastic based on
    consistency percentages.
    """
    attacker_consistency = strategy_consistency["attacker_consistency"]
    defender_consistency = strategy_consistency["defender_consistency"]
    
    # Sort attackers by consistency (ascending)
    sorted_attackers = sorted(
        attacker_consistency.items(),
        key=lambda x: x[1]["consistency_percentage"]
    )
    
    # Sort defenders by consistency (ascending)
    sorted_defenders = sorted(
        defender_consistency.items(),
        key=lambda x: x[1]["consistency_percentage"]
    )
    
    # Strategies with consistency below 50% are highly likely to be stochastic
    stochastic_attackers = [
        name for name, data in sorted_attackers 
        if data["consistency_percentage"] < 50 and data["total_runs"] > 10
    ]
    
    stochastic_defenders = [
        name for name, data in sorted_defenders 
        if data["consistency_percentage"] < 50 and data["total_runs"] > 10
    ]
    
    # Strategies with consistency 50-80% are moderately likely to be stochastic
    moderately_stochastic_attackers = [
        name for name, data in sorted_attackers 
        if 50 <= data["consistency_percentage"] < 80 and data["total_runs"] > 10
    ]
    
    moderately_stochastic_defenders = [
        name for name, data in sorted_defenders 
        if 50 <= data["consistency_percentage"] < 80 and data["total_runs"] > 10
    ]
    
    return {
        "highly_stochastic_attackers": stochastic_attackers,
        "highly_stochastic_defenders": stochastic_defenders,
        "moderately_stochastic_attackers": moderately_stochastic_attackers,
        "moderately_stochastic_defenders": moderately_stochastic_defenders,
        "sorted_attacker_consistency": sorted_attackers,
        "sorted_defender_consistency": sorted_defenders
    }

def analyze_strategy_pair_patterns(different_results):
    """
    Analyze patterns in the differences between strategy pairs.
    Look for consistent differences in payoff, time, tags, captures.
    """
    # Extract the differences by strategy pair
    pair_differences = defaultdict(list)
    
    for diff in different_results:
        filename = diff["filename"]
        
        # Extract strategies from the filename
        try:
            parts = filename.split("_")
            for i, part in enumerate(parts):
                if part == "attacker":
                    attacker = parts[i+1]
                if part == "defender":
                    defender = parts[i+1]
                    break
            
            strategy_pair = f"{attacker} vs {defender}"
            
            # Calculate the specific differences
            repo1 = diff["repo1"]
            repo2 = diff["repo2"]
            
            payoff_diff = abs(repo1.get("payoff", 0) - repo2.get("payoff", 0))
            time_diff = abs(repo1.get("time", 0) - repo2.get("time", 0))
            captures_diff = abs(repo1.get("total_captures", 0) - repo2.get("total_captures", 0))
            tags_diff = abs(repo1.get("total_tags", 0) - repo2.get("total_tags", 0))
            
            pair_differences[strategy_pair].append({
                "payoff_diff": payoff_diff,
                "time_diff": time_diff,
                "captures_diff": captures_diff,
                "tags_diff": tags_diff
            })
        except Exception:
            continue  # Skip if we can't parse the filename
    
    # Analyze the patterns in differences
    patterns = {}
    for pair, diffs in pair_differences.items():
        if len(diffs) < 5:  # Need enough samples
            continue
            
        avg_payoff_diff = sum(d["payoff_diff"] for d in diffs) / len(diffs)
        avg_time_diff = sum(d["time_diff"] for d in diffs) / len(diffs)
        avg_captures_diff = sum(d["captures_diff"] for d in diffs) / len(diffs)
        avg_tags_diff = sum(d["tags_diff"] for d in diffs) / len(diffs)
        
        patterns[pair] = {
            "sample_size": len(diffs),
            "avg_payoff_diff": avg_payoff_diff,
            "avg_time_diff": avg_time_diff,
            "avg_captures_diff": avg_captures_diff,
            "avg_tags_diff": avg_tags_diff
        }
    
    return patterns

def print_stochastic_analysis(analysis):
    """Print the stochastic analysis results in a readable format."""
    info("\n" + "=" * 50)
    info("Stochastic Strategy Analysis")
    info("=" * 50)
    
    # Print strategy consistency percentages
    info("\nAttacker Strategy Consistency (lower % = more stochastic):")
    attacker_data = analysis["strategy_consistency"]["attacker_consistency"]
    sorted_attackers = sorted(
        attacker_data.items(), 
        key=lambda x: x[1]["consistency_percentage"]
    )
    
    for attacker, data in sorted_attackers:
        consistency = data["consistency_percentage"]
        total = data["total_runs"]
        different = data["different_runs"]
        
        if consistency < 50:
            color = "red"  # Highly stochastic
        elif consistency < 80:
            color = "yellow"  # Moderately stochastic
        else:
            color = "green"  # Likely deterministic
            
        info(colored(
            f"  {attacker}: {consistency:.1f}% consistent "
            f"({different} different out of {total} runs)",
            color
        ))
    
    info("\nDefender Strategy Consistency (lower % = more stochastic):")
    defender_data = analysis["strategy_consistency"]["defender_consistency"]
    sorted_defenders = sorted(
        defender_data.items(), 
        key=lambda x: x[1]["consistency_percentage"]
    )
    
    for defender, data in sorted_defenders:
        consistency = data["consistency_percentage"]
        total = data["total_runs"]
        different = data["different_runs"]
        
        if consistency < 50:
            color = "red"  # Highly stochastic
        elif consistency < 80:
            color = "yellow"  # Moderately stochastic
        else:
            color = "green"  # Likely deterministic
            
        info(colored(
            f"  {defender}: {consistency:.1f}% consistent "
            f"({different} different out of {total} runs)",
            color
        ))
    
    # Print conclusion about which strategies are stochastic
    stochastic = analysis["stochastic_candidates"]
    
    info("\nHighly Likely Stochastic Strategies:")
    if stochastic["highly_stochastic_attackers"]:
        for attacker in stochastic["highly_stochastic_attackers"]:
            info(colored(f"  Attacker: {attacker}", "red"))
    else:
        info("  No highly stochastic attackers identified")
        
    if stochastic["highly_stochastic_defenders"]:
        for defender in stochastic["highly_stochastic_defenders"]:
            info(colored(f"  Defender: {defender}", "red"))
    else:
        info("  No highly stochastic defenders identified")
    
    info("\nModerately Likely Stochastic Strategies:")
    if stochastic["moderately_stochastic_attackers"]:
        for attacker in stochastic["moderately_stochastic_attackers"]:
            info(colored(f"  Attacker: {attacker}", "yellow"))
    else:
        info("  No moderately stochastic attackers identified")
        
    if stochastic["moderately_stochastic_defenders"]:
        for defender in stochastic["moderately_stochastic_defenders"]:
            info(colored(f"  Defender: {defender}", "yellow"))
    else:
        info("  No moderately stochastic defenders identified")
    
    # Print patterns in differences
    info("\nDifference Patterns by Strategy Pair:")
    patterns = analysis["strategy_pair_patterns"]
    for pair, data in sorted(patterns.items(), key=lambda x: x[1]["avg_payoff_diff"], reverse=True):
        info(f"  {pair} (sample size: {data['sample_size']}):")
        info(f"    Avg Payoff Diff: {data['avg_payoff_diff']:.2f}")
        info(f"    Avg Time Diff: {data['avg_time_diff']:.2f}")
        info(f"    Avg Captures Diff: {data['avg_captures_diff']:.2f}")
        info(f"    Avg Tags Diff: {data['avg_tags_diff']:.2f}")

# Example usage:
def identify_stochastic_strategies(data_path_1, data_path_2, comparison_results):
    """Identify which strategies are likely stochastic."""
    stochastic_analysis = analyze_stochastic_behavior(
        data_path_1, 
        data_path_2, 
        comparison_results["files"]["with_different_results"]
    )
    print_stochastic_analysis(stochastic_analysis)
    return stochastic_analysis

comparison_results = compare_repositories(DATA_PATH_1, DATA_PATH_2)
stochastic_analysis = identify_stochastic_strategies(DATA_PATH_1, DATA_PATH_2, comparison_results)

ℹ INFO: Starting comparison between repositories
ℹ INFO: Found 6400 JSON files in data/result/


Processing data/result/: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 6400/6400 [00:05<00:00, 1254.53file/s, Processing bdee3bca48]


ℹ INFO: Found 6400 JSON files in data/result-old/


Comparing common files: 100%|████████████████████████████████████████████████████| 6400/6400 [00:03<00:00, 1656.15file/s, Comparing log_815fb0dd95_attacker_MSU_Attacker_defender_Example_Defender.json]


ℹ INFO: ==================================================
ℹ INFO: Comparison Summary
ℹ INFO: ==================================================
ℹ INFO: 
Total files in Repo 1: 6400
ℹ INFO: Total files in Repo 2: 6400
ℹ INFO: Files only in Repo 1: 0
ℹ INFO: Files only in Repo 2: 0
ℹ INFO: Files with errors in Repo 1: 0
ℹ INFO: Files with errors in Repo 2: 0
ℹ INFO: Files with different results: 3991
ℹ INFO: Files with matching results: 2409
✓ SUCCESS: All files in Repo 1 exist in Repo 2
✓ SUCCESS: All files in Repo 2 exist in Repo 1
✓ SUCCESS: No files with errors in Repo 1
✓ SUCCESS: No files with errors in Repo 2
✗ ERROR: 
Found 3991 files with different results
✗ ERROR:   Different results: log_346e554198_attacker_GMU_Attacker_defender_UNCC_Defender.json
✗ ERROR:     Repo 1: payoff=-2.0, time=9, captures=2, tags=8
✗ ERROR:     Repo 2: payoff=0.0, time=9, captures=2, tags=4
✗ ERROR:   Different results: log_a6083ed2ca_attacker_GMU_Attacker_defender_Example_Defender.json
✗ ERROR:     

Analyzing strategy consistency: 100%|██████████████████████| 6400/6400 [00:00<00:00, 20605.74file/s]

ℹ INFO: 
ℹ INFO: Stochastic Strategy Analysis
ℹ INFO: ==================================================
ℹ INFO: 
Attacker Strategy Consistency (lower % = more stochastic):
ℹ INFO:   MSU_Attacker: 37.3% consistent (1003 different out of 1600 runs)
ℹ INFO:   Example_Attacker: 37.5% consistent (1000 different out of 1600 runs)
ℹ INFO:   GMU_Attacker: 37.9% consistent (994 different out of 1600 runs)
ℹ INFO:   UNCC_Attacker: 37.9% consistent (994 different out of 1600 runs)
ℹ INFO: 
Defender Strategy Consistency (lower % = more stochastic):
ℹ INFO:   MSU_Defender: 34.3% consistent (1051 different out of 1600 runs)
ℹ INFO:   UNCC_Defender: 38.7% consistent (980 different out of 1600 runs)
ℹ INFO:   GMU_Defender: 38.7% consistent (980 different out of 1600 runs)
ℹ INFO:   Example_Defender: 38.7% consistent (980 different out of 1600 runs)
ℹ INFO: 
Highly Likely Stochastic Strategies:
ℹ INFO:   Attacker: MSU_Attacker
ℹ INFO:   Attacker: Example_Attacker
ℹ INFO:   Attacker: GMU_Attacker
ℹ INF

In [ ]:
def analyze_config_consistency(data_path_1, data_path_2, different_results):
    """
    Analyze differences grouped by configuration hash keys to identify
    if differences are consistent within the same configuration.
    
    Args:
        data_path_1: Path to first repository
        data_path_2: Path to second repository
        different_results: List of dictionaries containing filenames with different results
    
    Returns:
        Dictionary with analysis of differences by hash key
    """
    info("Analyzing difference consistency by configuration hash key...")
    
    # Group differences by hash key
    hash_key_differences = defaultdict(list)
    
    for diff in different_results:
        filename = diff["filename"]
        
        # Extract hash key from filename
        try:
            hash_key = filename.split("_")[1]  # Extract the hash key portion
            
            # Calculate the specific differences
            repo1 = diff["repo1"]
            repo2 = diff["repo2"]
            
            payoff_diff = repo1.get("payoff", 0) - repo2.get("payoff", 0)
            time_diff = repo1.get("time", 0) - repo2.get("time", 0)
            captures_diff = repo1.get("total_captures", 0) - repo2.get("total_captures", 0)
            tags_diff = repo1.get("total_tags", 0) - repo2.get("total_tags", 0)
            
            # Extract strategies from filename for context
            attacker = None
            defender = None
            parts = filename.split("_")
            for i, part in enumerate(parts):
                if part == "attacker":
                    attacker = parts[i+1]
                if part == "defender":
                    defender = parts[i+1]
                    break
            
            hash_key_differences[hash_key].append({
                "filename": filename,
                "attacker": attacker,
                "defender": defender,
                "payoff_diff": payoff_diff,
                "time_diff": time_diff,
                "captures_diff": captures_diff,
                "tags_diff": tags_diff
            })
        except Exception as e:
            error(f"Error processing {filename} for hash key analysis: {str(e)}")
    
    # Analyze consistency within each hash key
    hash_key_consistency = {}
    
    for hash_key, diffs in hash_key_differences.items():
        if len(diffs) < 2:  # Need at least 2 samples to compare
            continue
            
        # Check if all differences are exactly the same
        consistent_payoff = len(set(d["payoff_diff"] for d in diffs)) == 1
        consistent_time = len(set(d["time_diff"] for d in diffs)) == 1
        consistent_captures = len(set(d["captures_diff"] for d in diffs)) == 1
        consistent_tags = len(set(d["tags_diff"] for d in diffs)) == 1
        
        # All metrics are consistent across all files with this hash key
        fully_consistent = consistent_payoff and consistent_time and consistent_captures and consistent_tags
        
        # Get unique values for each metric
        unique_payoff_diffs = sorted(list(set(d["payoff_diff"] for d in diffs)))
        unique_time_diffs = sorted(list(set(d["time_diff"] for d in diffs)))
        unique_captures_diffs = sorted(list(set(d["captures_diff"] for d in diffs)))
        unique_tags_diffs = sorted(list(set(d["tags_diff"] for d in diffs)))
        
        # Store consistency information
        hash_key_consistency[hash_key] = {
            "sample_size": len(diffs),
            "fully_consistent": fully_consistent,
            "consistent_payoff": consistent_payoff,
            "consistent_time": consistent_time,
            "consistent_captures": consistent_captures,
            "consistent_tags": consistent_tags,
            "unique_payoff_diffs": unique_payoff_diffs,
            "unique_time_diffs": unique_time_diffs,
            "unique_captures_diffs": unique_captures_diffs,
            "unique_tags_diffs": unique_tags_diffs,
            "examples": diffs[:5]  # Store a few examples for reference
        }
    
    return hash_key_consistency

def print_config_consistency_analysis(hash_key_consistency):
    """Print the hash key consistency analysis in a readable format."""
    info("\n" + "=" * 50)
    info("Configuration Hash Key Consistency Analysis")
    info("=" * 50)
    
    # Group by consistency
    fully_consistent_configs = []
    partially_consistent_configs = []
    inconsistent_configs = []
    
    for hash_key, data in hash_key_consistency.items():
        if data["fully_consistent"]:
            fully_consistent_configs.append((hash_key, data))
        elif any([data["consistent_payoff"], data["consistent_time"], 
                 data["consistent_captures"], data["consistent_tags"]]):
            partially_consistent_configs.append((hash_key, data))
        else:
            inconsistent_configs.append((hash_key, data))
    
    # Print fully consistent configs
    info(f"\nFully Consistent Configurations ({len(fully_consistent_configs)}):")
    if fully_consistent_configs:
        for hash_key, data in fully_consistent_configs:
            sample_size = data["sample_size"]
            example = data["examples"][0]
            info(colored(f"  Hash Key: {hash_key} (samples: {sample_size})", "green"))
            info(colored(f"    All diffs exactly match these values:", "green"))
            info(colored(f"    Payoff Diff: {example['payoff_diff']}", "green"))
            info(colored(f"    Time Diff: {example['time_diff']}", "green"))
            info(colored(f"    Captures Diff: {example['captures_diff']}", "green"))
            info(colored(f"    Tags Diff: {example['tags_diff']}", "green"))
    else:
        info("  None found")
    
    # Print partially consistent configs
    info(f"\nPartially Consistent Configurations ({len(partially_consistent_configs)}):")
    if partially_consistent_configs:
        for hash_key, data in partially_consistent_configs:
            sample_size = data["sample_size"]
            info(colored(f"  Hash Key: {hash_key} (samples: {sample_size})", "yellow"))
            
            # Print which metrics are consistent
            consistent_metrics = []
            if data["consistent_payoff"]:
                consistent_metrics.append(f"Payoff: {data['unique_payoff_diffs'][0]}")
            if data["consistent_time"]:
                consistent_metrics.append(f"Time: {data['unique_time_diffs'][0]}")
            if data["consistent_captures"]:
                consistent_metrics.append(f"Captures: {data['unique_captures_diffs'][0]}")
            if data["consistent_tags"]:
                consistent_metrics.append(f"Tags: {data['unique_tags_diffs'][0]}")
            
            info(colored(f"    Consistent metrics: {', '.join(consistent_metrics)}", "yellow"))
            
            # Print which metrics vary
            if not data["consistent_payoff"]:
                values = data["unique_payoff_diffs"]
                info(colored(f"    Payoff varies: {values}", "yellow"))
            if not data["consistent_time"]:
                values = data["unique_time_diffs"]
                info(colored(f"    Time varies: {values}", "yellow"))
            if not data["consistent_captures"]:
                values = data["unique_captures_diffs"]
                info(colored(f"    Captures varies: {values}", "yellow"))
            if not data["consistent_tags"]:
                values = data["unique_tags_diffs"]
                info(colored(f"    Tags varies: {values}", "yellow"))
    else:
        info("  None found")
    
    # Print inconsistent configs (only a few examples)
    info(f"\nInconsistent Configurations ({len(inconsistent_configs)}):")
    if inconsistent_configs:
        for hash_key, data in inconsistent_configs[:5]:  # Show only first 5
            sample_size = data["sample_size"]
            info(colored(f"  Hash Key: {hash_key} (samples: {sample_size})", "red"))
            info(colored(f"    Payoff varies: {data['unique_payoff_diffs'][:5]}" + 
                        ("..." if len(data['unique_payoff_diffs']) > 5 else ""), "red"))
            info(colored(f"    Time varies: {data['unique_time_diffs'][:5]}" + 
                        ("..." if len(data['unique_time_diffs']) > 5 else ""), "red"))
            info(colored(f"    Captures varies: {data['unique_captures_diffs'][:5]}" + 
                        ("..." if len(data['unique_captures_diffs']) > 5 else ""), "red"))
            info(colored(f"    Tags varies: {data['unique_tags_diffs'][:5]}" + 
                        ("..." if len(data['unique_tags_diffs']) > 5 else ""), "red"))
        
        if len(inconsistent_configs) > 5:
            info(colored(f"  ... and {len(inconsistent_configs) - 5} more", "red"))
    else:
        info("  None found")
    
    # Print overall stats
    info("\nOverall Statistics:")
    total_configs = len(hash_key_consistency)
    info(f"  Total unique configurations analyzed: {total_configs}")
    info(f"  Fully consistent: {len(fully_consistent_configs)} ({100*len(fully_consistent_configs)/total_configs:.1f}%)")
    info(f"  Partially consistent: {len(partially_consistent_configs)} ({100*len(partially_consistent_configs)/total_configs:.1f}%)")
    info(f"  Inconsistent: {len(inconsistent_configs)} ({100*len(inconsistent_configs)/total_configs:.1f}%)")

# Add this function to your main analysis:
def analyze_hash_key_consistency(data_path_1, data_path_2, comparison_results):
    """Analyze consistency of differences by configuration hash key."""
    hash_key_consistency = analyze_config_consistency(
        data_path_1, 
        data_path_2, 
        comparison_results["files"]["with_different_results"]
    )
    print_config_consistency_analysis(hash_key_consistency)
    return hash_key_consistency

# Usage in your notebook:
hash_key_analysis = analyze_hash_key_consistency(DATA_PATH_1, DATA_PATH_2, comparison_results)

ℹ INFO: Analyzing difference consistency by configuration hash key...
ℹ INFO: 
ℹ INFO: Configuration Hash Key Consistency Analysis
ℹ INFO: ==================================================
ℹ INFO: 
Fully Consistent Configurations (142):
ℹ INFO:   Hash Key: 32d9d80f7a (samples: 16)
ℹ INFO:     All diffs exactly match these values:
ℹ INFO:     Payoff Diff: -0.5
ℹ INFO:     Time Diff: 0
ℹ INFO:     Captures Diff: 1
ℹ INFO:     Tags Diff: 3
ℹ INFO:   Hash Key: 2f22b3e75b (samples: 16)
ℹ INFO:     All diffs exactly match these values:
ℹ INFO:     Payoff Diff: -0.5
ℹ INFO:     Time Diff: 0
ℹ INFO:     Captures Diff: 0
ℹ INFO:     Tags Diff: 1
ℹ INFO:   Hash Key: c7b4928d14 (samples: 16)
ℹ INFO:     All diffs exactly match these values:
ℹ INFO:     Payoff Diff: 1.0
ℹ INFO:     Time Diff: 0
ℹ INFO:     Captures Diff: 2
ℹ INFO:     Tags Diff: 2
ℹ INFO:   Hash Key: 634b87532a (samples: 16)
ℹ INFO:     All diffs exactly match these values:
ℹ INFO:     Payoff Diff: -1.0
ℹ INFO:     Time Diff: 0
ℹ